<a href="https://colab.research.google.com/github/tiennvuit/CS106.K21.KHTN/blob/master/18521489_NguyenVanTien_ValueIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đánh giá thuật toán Value Iteration



## Cài đặt thuật toán dựa trên source có sẵn

In [0]:
import numpy as np
import gym

In [0]:
"""
Original file is located at
    https://colab.research.google.com/drive/1ivUq_NzgIs3bXHN4mn6zfhg2VXs2uNtv
"""

class PlayGame():
    def __init__(self, name: str, max_iters=3, gamma=0.9):
        self.env = gym.make(name)
        self.max_iters = max_iters
        self.gamma = gamma

    def value_iteration(self):
        v_values = np.zeros(self.env.observation_space.n)
        for i in range(self.max_iters):
            prev_v_values = np.copy(v_values)

            # Compute value for each state
            for state in range(self.env.observation_space.n):
                q_values = []

                # Compute q-value for each action
                for action in range(self.env.action_space.n):                
                    q_value = 0
                    for prob, next_state, reward, done in self.env.P[state][action]:
                        q_value += prob * (reward + self.gamma * prev_v_values[next_state])
                    q_values.append(q_value)
                
                # Select the best action
                best_action = np.argmax(np.asarray(q_values))
                v_values[state] = q_values[best_action]
            
            # Check convergence
            if np.all(np.isclose(v_values, prev_v_values)):
                print('Converged at {}-th iteration.'.format(i))
                break
        
        return v_values

    def policy_extraction(self):
        v_values = self.value_iteration()
        policy = np.zeros(self.env.observation_space.n, dtype=np.int)
        
        # Compute the best action for each state in the game
        # Compute q-values for each (state-action) pair in the game
        for state in range(self.env.observation_space.n):
            q_values = []

            # Compute q-values for each action
            for action in range(self.env.action_space.n):
                q_value = 0
                for prob, next_state, reward, done in self.env.P[state][action]:
                    q_value += prob * (reward + self.gamma * v_values[next_state])
                q_values.append(q_value)

            # Select the best action
            best_action = np.argmax(np.asarray(q_values))
            policy[state] = best_action
        
        return policy


    def play(self):
        policy = self.policy_extraction()
        state = self.env.reset()
        steps = 0
        done = False
        while not done:
            action = policy[state]
            next_state, reward, done, info = self.env.step(action)
            steps += 1
            state = next_state

        return (reward, steps)

    def play_multiple_times(self):
        num_episodes = 1000
        list_of_steps = []
        num_failures = 0
        
        for i in range(num_episodes):
            reward, steps = self.play()
            if reward == 1:
                list_of_steps.append(steps)
            else:
                num_failures += 1

        print('# failures: {}/{}'.format(num_failures, num_episodes))
        print('avg. # steps: {}'.format(np.mean(list_of_steps)))

In [3]:
FrozenLake_v0 = PlayGame("FrozenLake-v0", max_iters=20, gamma=0.8)
FrozenLake_v0.play_multiple_times()

# failures: 538/1000
avg. # steps: 29.227272727272727


In [4]:
FrozenLake8x8_v0 = PlayGame("FrozenLake8x8-v0", max_iters=20, gamma=0.8)
FrozenLake8x8_v0.play_multiple_times()

# failures: 377/1000
avg. # steps: 72.58587479935794


In [5]:
Taxi_v3 = PlayGame("FrozenLake8x8-v0", max_iters=20, gamma=0.8)
Taxi_v3.play_multiple_times()

# failures: 373/1000
avg. # steps: 70.17543859649123


## Một số nhận sét về thuật toán Value Iteration:
- Ưu điểm:
    - Độ phức tạp của mỗi vòng lặp là $O(S^2A)$.
    - Sẽ hội tụ đến điểm tối ưu.
    - Value iteration is good for a small set of states because we will avoid computing very deep expectimax trees which run in exponential time.
    - Thuật toán này phù hợp cho những bài toán không gian trạng thái nhỏ vì chúng ta sẽ tránh được việc tính toán lớn đi duyệt sâu xuống câu expectimax - thời gian thực thi là hàm mũ.

- Nhược điểm:
    - Thuật toán này phải duyệt qua mọi trạng thái trong mỗi lần lặp và vì vậy nếu chúng ta có không gian trạng thái lớn  việc lặp lại giá trị phải tiêu tốn rất nhiều thời gian.
    - The "max" at a state rarely changes. This means that the relative size of the Q values converge well before the values converge.

## Tham khảo:
- [Value Iteration](https://cs188ai.fandom.com/wiki/Value_Iteration)
